In [2]:
import pandas as pd
from bs4 import BeautifulSoup
import requests as req

In [3]:
resp = req.get("https://dnd.su/spells/")
soup = BeautifulSoup(resp.text, 'lxml')

In [74]:
df = pd.DataFrame()
tags = []
hrefs = []
titles = []
levels = []
schools = []
for tag in soup.find_all("li", {"class": "for_filter"}):
    href = tag.find("a").get('href')
    title = tag.find("a").get('title')
    level = tag.find("span", {"class": "icon-card_level"}).getText()
    schls = []
    for i in [1, 2, 3, 4, 5, 6, 7, 8]:
        try:
            school = tag.find("span", {"class": f"icon-spells_school-{i}"}).get('title')
            schls.append(school)
        except:
            continue
    tags.append(str(tag))
    hrefs.append(str(f'https://dnd.su{href}'))
    titles.append(str(title))
    levels.append(level)
    schools.append(schls)
df['tags']=tags
df['hrefs']=hrefs
df['titles']=titles
df['levels']=levels
df['schools']=schools
df.loc[df['tags'].str.contains('Ритуал'), 'Ритуал'] = 1
df.loc[df['tags'].str.contains('Концентрация'), 'Концентрация'] = 1
df.loc[df['tags'].str.contains('Вербальный'), 'В'] = 1
df.loc[df['tags'].str.contains('Соматический'), 'С'] = 1
df.loc[df['tags'].str.contains('Материальный'), 'М'] = 1
print(df['tags'][0])
df.head(3)


<li class="for_filter" data-id="1" data-title_en="Hellish rebuke"><span class="icon-card_level text-in-icon" title="Заклинание 1-го уровня">1</span><span class="icon-spells_school-1" title="воплощение"></span><a href="/spells/1-hellish_rebuke/" title="Адское возмездие [Hellish rebuke]">Адское возмездие</a> <span class="end_icons"><span class="end_icon_component" title="Вербальный">В</span><span class="end_icon_component" title="Соматический">С</span></span></li>


,tags,hrefs,titles,levels,schools,Ритуал,Концентрация,В,С,М
0,"<li class=""for_filter"" data-id=""1"" data-title_...",https://dnd.su/spells/1-hellish_rebuke/,Адское возмездие [Hellish rebuke],1,[воплощение],NaN,NaN,1.0,1.0,NaN
1,"<li class=""for_filter"" data-id=""2"" data-title_...",https://dnd.su/spells/2-antipathy_sympathy/,Антипатия/симпатия [Antipathy/sympathy],8,[очарование],NaN,NaN,1.0,1.0,1.0
2,"<li class=""for_filter"" data-id=""3"" data-title_...",https://dnd.su/spells/3-aura_of_vitality/,Аура живучести [Aura of vitality],3,[воплощение],NaN,1.0,1.0,NaN,NaN


In [176]:
descriptions = pd.DataFrame()
# hrefs = ['https://dnd.su/spells/1-hellish_rebuke/', 'https://dnd.su/spells/2-antipathy_sympathy/', 'https://dnd.su/spells/3-aura_of_vitality/']
hrefs = df['hrefs'].unique()
for href in hrefs:
    resp = req.get(href)
    soup = BeautifulSoup(resp.text, 'lxml')
    descr_df = pd.DataFrame()
    chars = []
    txts = []
    for tag in soup.find_all("div", {"class": "card-body"}):
        for str in tag.find_all("li"):
            try:
                char = str.find("strong").getText()
            except:
                char=''
            txt = str.getText()
            chars.append(char)
            txts.append(txt)
    descr_df['chars'] = chars
    descr_df['txts'] = txts
    descr_df = descr_df.append(pd.DataFrame({
        'chars':['Ссылка'],
        'txts':[href]
    }))
    descr_df = descr_df.append(pd.DataFrame({
        'chars':['Описание'],
        'txts':[tag.find("div").get_text()]
    }))
    descr_df = descr_df.transpose()
    descr_df = descr_df.reset_index(drop=True)
    descr_df.columns = descr_df.iloc[0]
    descr_df = descr_df[1:]
    for i in descr_df.columns:
        if len(i)==0:
            try:
                descr_df = descr_df.drop(columns=[i])
            except:
                descr_df = descr_df
        else:
            descr_df = descr_df
    descriptions = pd.concat([descriptions, descr_df])

descriptions.head(5)


,Время накладывания:,Дистанция:,Компоненты:,Длительность:,Классы:,Архетипы:,Источник:,На больших уровнях.,Ссылка,Описание,...,«Связанный предмет»,«Очень знакомое»,«Изредка видимое»,«Виденное один раз»,«Описанное»,«Ложно описанное»,Увеличение,Бычья сила.,Хронургия.,Благословение воды
1,"Время накладывания: 1 реакция, совершаемая вам...",Дистанция: 60 футов,"Компоненты: В, С",Длительность: Мгновенная,Классы: колдун,Архетипы: Клятвопреступник (паладин),Источник: «Player's handbook»,"Вы указываете пальцем, и существо, причинившее...",https://dnd.su/spells/1-hellish_rebuke/,"Вы указываете пальцем, и существо, причинившее...",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Время накладывания: 1 час,Дистанция: 60 футов,"Компоненты: В, С, М (пропитанный уксусом кусо...",Длительность: 10 дней,"Классы: бард (TCoE), волшебник, друид",NaN,Источник: «Player's handbook»,NaN,https://dnd.su/spells/2-antipathy_sympathy/,Это заклинание привлекает или отталкивает выбр...,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Время накладывания: 1 действие,Дистанция: На себя (30-футовый радиус),Компоненты: В,"Длительность: Концентрация, вплоть до 1 минуты","Классы: друид (TCoE), жрец (TCoE), паладин","Архетипы: Боевой кузнец (изобретатель), Домен ...",Источник: «Player's handbook»,NaN,https://dnd.su/spells/3-aura_of_vitality/,От вас исходит аура живительной энергии с ради...,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Время накладывания: 1 действие,Дистанция: На себя (30-футов радиус),Компоненты: В,"Длительность: Концентрация, вплоть до 10 минут","Классы: жрец (TCoE), паладин","Архетипы: Бессмертный (колдун), Круг дикого ог...",Источник: «Player's handbook»,NaN,https://dnd.su/spells/4-aura_of_life/,От вас исходит защищающая жизнь аура с радиусо...,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Время накладывания: 1 действие,Дистанция: На себя (30-футов радиус),Компоненты: В,"Длительность: Концентрация, вплоть до 10 минут","Классы: жрец (TCoE), паладин","Архетипы: Боевой кузнец (изобретатель), Домен ...",Источник: «Player's handbook»,NaN,https://dnd.su/spells/5-aura_of_purity/,От вас исходит очищающая аура с радиусом 30 фу...,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [236]:

df_new = df.merge(descriptions, left_on='hrefs', right_on='Ссылка', how='inner')
print(f'len df {len(df)} \n len descriptions {len(descriptions)} \n len df_new {len(df_new)}')

len df 516 
 len descriptions 516 
 len df_new 516


In [237]:
df_new= df_new[['tags', 'hrefs', 'titles', 'levels', 'schools', 'Ритуал',
       'Концентрация', 'В', 'С', 'М', 'Время накладывания:', 'Дистанция:',
       'Компоненты:', 'Длительность:', 'Классы:', 'Архетипы:', 'Источник:',
       'Ссылка', 'Описание']]

for column in ['Время накладывания:', 'Дистанция:',
       'Компоненты:', 'Длительность:', 'Классы:', 'Архетипы:', 'Источник:']:
    for i in range(0, len(df_new)):
        try:
            df_new[column][i] = df_new[column][i].replace(column, '')
        except:
            df_new[column][i] = df_new[column][i]

df_new.head(5)

/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  # Remove the CWD from sys.path while we load stuff.
/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  if sys.path[0] == '':


,tags,hrefs,titles,levels,schools,Ритуал,Концентрация,В,С,М,Время накладывания:,Дистанция:,Компоненты:,Длительность:,Классы:,Архетипы:,Источник:,Ссылка,Описание
0,"<li class=""for_filter"" data-id=""1"" data-title_...",https://dnd.su/spells/1-hellish_rebuke/,Адское возмездие [Hellish rebuke],1,[воплощение],NaN,NaN,1.0,1.0,NaN,"1 реакция, совершаемая вами в ответ на получе...",60 футов,"В, С",Мгновенная,колдун,Клятвопреступник (паладин),«Player's handbook»,https://dnd.su/spells/1-hellish_rebuke/,"Вы указываете пальцем, и существо, причинившее..."
1,"<li class=""for_filter"" data-id=""2"" data-title_...",https://dnd.su/spells/2-antipathy_sympathy/,Антипатия/симпатия [Antipathy/sympathy],8,[очарование],NaN,NaN,1.0,1.0,1.0,1 час,60 футов,"В, С, М (пропитанный уксусом кусок квасцов д...",10 дней,"бард (TCoE), волшебник, друид",NaN,«Player's handbook»,https://dnd.su/spells/2-antipathy_sympathy/,Это заклинание привлекает или отталкивает выбр...
2,"<li class=""for_filter"" data-id=""3"" data-title_...",https://dnd.su/spells/3-aura_of_vitality/,Аура живучести [Aura of vitality],3,[воплощение],NaN,1.0,1.0,NaN,NaN,1 действие,На себя (30-футовый радиус),В,"Концентрация, вплоть до 1 минуты","друид (TCoE), жрец (TCoE), паладин","Боевой кузнец (изобретатель), Домен сумерек (...",«Player's handbook»,https://dnd.su/spells/3-aura_of_vitality/,От вас исходит аура живительной энергии с ради...
3,"<li class=""for_filter"" data-id=""4"" data-title_...",https://dnd.su/spells/4-aura_of_life/,Аура жизни [Aura of life],4,[ограждение],NaN,1.0,1.0,NaN,NaN,1 действие,На себя (30-футов радиус),В,"Концентрация, вплоть до 10 минут","жрец (TCoE), паладин","Бессмертный (колдун), Круг дикого огня (друид)",«Player's handbook»,https://dnd.su/spells/4-aura_of_life/,От вас исходит защищающая жизнь аура с радиусо...
4,"<li class=""for_filter"" data-id=""5"" data-title_...",https://dnd.su/spells/5-aura_of_purity/,Аура очищения [Aura of purity],4,[ограждение],NaN,1.0,1.0,NaN,NaN,1 действие,На себя (30-футов радиус),В,"Концентрация, вплоть до 10 минут","жрец (TCoE), паладин","Боевой кузнец (изобретатель), Домен мира (жре...",«Player's handbook»,https://dnd.su/spells/5-aura_of_purity/,От вас исходит очищающая аура с радиусом 30 фу...


In [238]:
df_new.to_excel('Spells from dndsu.xlsx')